# Fetch Data If It Doesn't Exist

Data from: https://www.kaggle.com/datasets/samuelcortinhas/muffin-vs-chihuahua-image-classification

In [ ]:
import os
from pathlib import Path

import kagglehub
import optuna

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

import kagglehub

# Download latest version
path = kagglehub.dataset_download("samuelcortinhas/muffin-vs-chihuahua-image-classification")

print("Path to dataset files:", path)


Path to dataset files: C:\Users\JTSoundy1\.cache\kagglehub\datasets\samuelcortinhas\muffin-vs-chihuahua-image-classification\versions\2
